# Non-Intrusive Load Monitoring Framework

# Data

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd drive/My Drive/Saya

/content/drive/My Drive/Saya


In [0]:
ts = pd.read_excel('data/FebMarApr.xlsx')

In [0]:
ts.head()

,CreatedDate,SupplyWaterTemperature,SupplyWaterPressure,BackWaterTemperature,BackWaterPressure,ValveStatus,TotalFlowTotalFlow,Flow,FlowRate,SendInterval
0,2019-02-20 08:00:00,56.084,100.004,0,0,1,8807.76,0.0,0.0,60
1,2019-02-20 08:01:00,56.084,100.001,0,0,1,8807.76,0.0,0.0,60
2,2019-02-20 08:02:00,56.084,100.001,0,0,1,8807.76,0.0,0.0,60
3,2019-02-20 08:03:00,56.084,100.001,0,0,1,8807.76,0.0,0.0,60
4,2019-02-20 08:04:00,54.734,100.002,0,0,1,8807.76,0.0,0.0,60


# Time Series segmentation

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')

In [0]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

matrix = []

for ts_i in window(ts.SupplyWaterPressure, 30):
    matrix.append(ts_i)

matrix = np.array(matrix)
matrix.shape

(58170, 30)

In [0]:
dates_matrix = []

for time in window(ts.index, 30):
    dates_matrix.append(time)
    
dates_matrix = np.array(dates_matrix)
dates_matrix.shape

(58170, 30)

# Make Labels for Segmented Time Series Data According to Given Information




*   Shower (Label 1) - This is the smallest amplitude repeating pulse at ??

*   Toilet (Label 2) - Runs for a duration of about ?? Starts at the same time everyday.

*   Faucet 1 (Label 3) - The most common repeating pulse. (At about ?? amplitude and a width of about ??minutes)
*   
Faucet 2 (Label 4) - Another repeating pulse but less frequent  (At about ?? amplitude and a width of about ??minutes)





In [0]:
labels = []
for ts_i in matrix:
    delta_ts_i = np.diff(ts_i)
    if all(ts_i < 80): #Shower
        label = 1
    elif all((ts_i > 85) & (ts_i < 80)): #Toilets
        label = 2
    elif all((ts_i > 90) & (ts_i < 85)): #Faucet 1
        label = 3
    elif any(ts_i > 90): #Faucet 2
        label = 4
    else: #Other Appliances (Unknown)
        label = 5
    labels.append(label)
labels = np.array(labels)

In [0]:

from collections import Counter
d = Counter(labels)
print (d)

Counter({4: 58170})


# Supervised Learning: Time Series K-Nearest Neighbor Classification

In [0]:
# from sklearn.metrics import classification_report

class ts_KnnClassifier(object):
    def __init__(self):
        '''
        preds is a list of predictions that will be made.
        plotter indicates whether to plot each nearest neighbor as it is found.
        '''
        self.preds=[]

    def predict(self, X_train, X_test, y_train, w, progress=False):
        '''
        1-nearest neighbor classification algorithm using LB_Keogh lower 
        bound as similarity measure. Option to use DTW distance instead
        but is much slower.
        '''
        for i, s1 in enumerate(X_test):
            if progress and i % 1000 == 0:
                print (str(i+1) + ' points classified')
            min_dist = float('inf')
            closest_seq_ind = []

            for j, s2 in enumerate(X_train):
                if self.LB_Keogh(s1, s2, 5) < min_dist:
                    dist = self.DTWDistance(s1, s2, w)
                    if dist < min_dist:
                        min_dist = dist
                        closest_seq_ind = j
            self.preds.append(y_train[closest_seq_ind])

    def performance(self, true_results):
        '''
        If the actual test set labels are known, can determine classification
        accuracy.
        '''
        return classification_report(true_results, self.preds)

    def get_preds(self):
        return self.preds

    def DTWDistance(self, s1, s2, w=None):
        '''
        Calculates dynamic time warping Euclidean distance between two
        sequences. Option to enforce locality constraint for window w.
        '''
        DTW={}

        if w:
            w = max(w, abs(len(s1)-len(s2)))

            for i in range(-1,len(s1)):
                for j in range(-1,len(s2)):
                    DTW[(i, j)] = float('inf')

        else:
            for i in range(len(s1)):
                DTW[(i, -1)] = float('inf')
            for i in range(len(s2)):
                DTW[(-1, i)] = float('inf')

        DTW[(-1, -1)] = 0

        for i in range(len(s1)):
            if w:
                for j in range(max(0, i-w), min(len(s2), i+w)):
                    dist= (s1[i] - s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)], DTW[(i, j-1)], DTW[(i-1, j-1)])
            else:
                for j in range(len(s2)):
                    dist= (s1[i] - s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)], DTW[(i, j-1)], DTW[(i-1, j-1)])

        return np.sqrt(DTW[len(s1)-1, len(s2)-1])

    def LB_Keogh(self,s1,s2,r):
        '''
        Calculates LB_Keough lower bound to dynamic time warping. Linear
        complexity compared to quadratic complexity of dtw.
        '''
        LB_sum=0
        for ind, i in enumerate(s1):

            lower_bound=min(s2[(ind-r if ind-r >= 0 else 0):(ind + r)])
            upper_bound=max(s2[(ind-r if ind-r >= 0 else 0):(ind + r)])

            if i > upper_bound:
                LB_sum = LB_sum + (i - upper_bound)**2
            elif i < lower_bound:
                LB_sum = LB_sum + (i - lower_bound)**2

        return np.sqrt(LB_sum)


In [0]:
from sklearn.metrics import f1_score, precision_score, recall_score

y_predsCV = []

def performTimeSeriesCV(X_train, y_train, number_folds):
    """
    Given X_train and y_train (the test set is excluded from the Cross Validation),
    number of folds, the ML algorithm to implement and the parameters to test,
    the function acts based on the following logic: it splits X_train and y_train in a
    number of folds equal to number_folds. Then train on one fold and tests accuracy
    on the consecutive as follows:
    - Train on fold 1, test on 2
    - Train on fold 1-2, test on 3
    - Train on fold 1-2-3, test on 4
    ....
    Returns mean of test accuracies.
    """
    global y_predsCV
 
    print ('Size train set: ', X_train.shape)
    
    # k is the size of each fold. It is computed dividing the number of 
    # rows in X_train by number_folds. This number is floored and coerced to int
    k = int(np.floor(float(X_train.shape[0]) / number_folds))
    print ('Size of each fold: ', k)
    
    # initialize to zero the accuracies array. It is important to stress that
    # in the CV of Time Series if I have n folds I test n-1 folds as the first
    # one is always needed to train
    f1_scores = np.zeros(number_folds-1)
    precision_scores = np.zeros(number_folds-1)
    recall_scores = np.zeros(number_folds-1)
    
 
    # loop from the first 2 folds to the total number of folds    
    for i in range(2, number_folds + 1):
        print ('')
        
        # the split is the percentage at which to split the folds into train
        # and test. For example when i = 2 we are taking the first 2 folds out 
        # of the total available. In this specific case we have to split the
        # two of them in half (train on the first, test on the second), 
        # so split = 1/2 = 0.5 = 50%. When i = 3 we are taking the first 3 folds 
        # out of the total available, meaning that we have to split the three of them
        # in two at split = 2/3 = 0.66 = 66% (train on the first 2 and test on the
        # following)
        split = float(i-1)/i
        
        # example with i = 4 (first 4 folds):
        #      Splitting the first       4        chunks at          3      /        4
        print ('Splitting the first ' + str(i) + ' chunks at ' + str(i-1) + '/' + str(i) )
        
        # as we loop over the folds X and y are updated and increase in size.
        # This is the data that is going to be split and it increases in size 
        # in the loop as we account for more folds. If k = 300, with i starting from 2
        # the result is the following in the loop
        # i = 2
        # X = X_train[:(600)]
        # y = y_train[:(600)]
        #
        # i = 3
        # X = X_train[:(900)]
        # y = y_train[:(900)]
        # .... 
        X = X_train[:(k*i)]
        y = y_train[:(k*i)]
        print ('Size of train + test: ', X.shape) # the size of the dataframe is going to be k*i
 
        # X and y contain both the folds to train and the fold to test.
        # index is the integer telling us where to split, according to the
        # split percentage we have set above
        index = int(np.floor(X.shape[0] * split))
        
        # folds used to train the model        
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]
        
        # fold used to test the model
        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]
        
        #Classify Test Folds
        ts_knn = ts_KnnClassifier()
        print ('')
        print ('KNN Classifier Is Classifying Data Points')
        ts_knn.predict(X_trainFolds, X_testFold, y_trainFolds, w = 10, progress=True)
        predictions = ts_knn.get_preds()
        y_predsCV.extend(predictions)
        
        #
        
        # i starts from 2 so the zeroth element in accuracies array is i-2. performClassification()
        #is a function which takes care of a classification problem. This is only an example and 
        #you can replace this function with whatever ML approach you need.
        f1_scores[i-2] = f1_score(y_testFold, predictions, average='weighted')
        precision_scores[i-2] =  precision_score(y_testFold, predictions, average='weighted')
        recall_scores[i-2] = recall_score(y_testFold, predictions, average='weighted')
        
        
#         # example with i = 4:
#         #      Accuracy on fold         4     :    0.85423
        print ('F1 Score on fold ' + str(i) + ': ', f1_scores[i-2])
        print ('Precision Score on fold ' + str(i) + ': ', precision_scores[i-2])
        print ('Recall Score on fold ' + str(i) + ': ', recall_scores[i-2])
    
    # the function returns the means of the precision_scores, recall scores, and f1_scores on the n-1 folds    
    return precision_scores.mean(), recall_scores.mean(), f1_scores.mean()

In [0]:
X_trainFolds, y_trainFolds, X_testFold, y_testFold = performTimeSeriesCV(matrix, labels, 10)

Size train set:  (58170, 30)
Size of each fold:  5817

Splitting the first 2 chunks at 1/2
Size of train + test:  (11634, 30)

KNN Classifier Is Classifying Data Points
1 points classified
1001 points classified
2001 points classified


In [0]:

ts_knn = ts_KnnClassifier()
ts_knn.predict(X_trainFolds, X_testFold, y_trainFolds, w = 10, progress=True)
y_pred = ts_knn.get_preds()

In [0]:
d1 = Counter(y_pred)
print d1

In [0]:
d2 = Counter(y_testFold)
print d2

In [0]:
print ts_knn.performance(y_testFold)

In [0]:
number_folds = 10
k = int(np.floor(float(matrix.shape[0]) / number_folds))
split = float(2-1)/2
X1 = matrix[:(k*2)]
y1 = labels[:(k*2)]
index = int(np.floor(X1.shape[0] * split))
X_trainFolds1 = X1[:index]        
y_trainFolds1 = y1[:index]
X_testFold1 = X1[(index + 1):]
y_testFold1 = y1[(index + 1):]

In [0]:
from joblib import Parallel, delayed
from copy_reg import pickle
from types import MethodType

def _pickle_method(method):
    func_name = method.im_func.__name__
    obj = method.im_self
    cls = method.im_class
    return _unpickle_method, (func_name, obj, cls)

def _unpickle_method(func_name, obj, cls):
    for cls in cls.mro():
        try:
            func = cls.__dict__[func_name]
        except KeyError:
            pass
        else:
            break
    return func.__get__(obj, cls)

pickle(MethodType, _pickle_method, _unpickle_method)
Parallel(n_jobs=4)(delayed(ts_knn.predict)(X_trainFolds1, testInstance, y_trainFolds1) for testInstance in X_testFold1)

In [0]:
def generateApplianceTimeSeries(X_testFold, y_pred):
    appliances_ts = [[] for _ in range(1,6)]

    #Generate Time Series for Refrigerator Appliance
    label = 1
    refrigerator_sub = np.array(X_testFold, copy=True)
    replacement = np.ones(shape=refrigerator_sub[0].shape)* 300

    #Randomnly Sample From A Collection of Refrigerator-Only
    #Time Series Segments and assign it to the occurences where
    #Not Only A Refrigerator Is On
    for i in np.nditer(np.where(y_pred != label)[0]):
        arr = refrigerator_sub[y_pred == label]
        index_array = np.arange(arr.shape[0])
        np.random.shuffle(index_array)
        replacement = arr[index_array[0]]
        refrigerator_sub[i] = replacement

    refrigerator_ts = []
    for i in range(len(refrigerator_sub)):
        if i == 0:
            refrigerator_ts.extend(list(refrigerator_sub[i]))
        else:
            refrigerator_ts.append(refrigerator_sub[i][-1])

    appliances_ts[label - 1] = refrigerator_ts

    #Generate Time Series for Non-refrigerator Appliances
    for label in range(2,6):
        appliance_sub = np.array(X_testFold, copy=True)
        replacement = np.zeros(shape=appliance_sub[0].shape)
        appliance_sub[y_pred != label] = replacement

        appliance_ts = []
        for i in range(len(appliance_sub)):
            if i == 0:
                appliance_ts.extend(list(appliance_sub[i]))
            else:
                appliance_ts.append(appliance_sub[i][-1])
        appliances_ts[label - 1] = appliance_ts

    # for ts_i in appliances_ts:
    #     print ts_i
    
    return appliances_ts

In [0]:
appliances_ts = generateApplianceTimeSeries(X_testFold, y_pred)


In [0]:
def convertAppliancesTimeSeriesToDataFrames(appliances_ts, tmpstmp, index=-4486):
    appliances = ['Central AC1 (W)', 'Central AC2 (W)', 'Pool Pump (W)', 'Refrigerator (W)']
    labels = [2, 3, 1, 0]
    ts_dfs = []

    for idx, label in enumerate(labels):
        s = pd.DataFrame(appliances_ts[label])
        s[appliances[idx]] = s[0]
        s['timestamp'] = tmpstmp[::60][index:]
        s['timestamp'] = pd.to_datetime(s['timestamp'],unit='s')
        s.drop(0,axis=1,inplace=True)
        s.set_index(keys='timestamp', inplace=True)
        s = s.tz_localize('UTC').tz_convert('US/Pacific')
        ts_dfs.append(s)
    return ts_dfs

In [0]:
ts_dfs = convertAppliancesTimeSeriesToDataFrames(appliances_ts, tmpstmp)

# Data Visualiztion of Water Consumption Time Series for Individual Applications

In [0]:
def plotDisAggregatedTimeSeries(ts_dfs):
    fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(20,16))
    nrow = 0
    for s in ts_dfs:
        ax=axes[nrow]
        s.plot(color='red', ax=ax, alpha=0.55)
        ax.set_ylabel('Watts (W)')
        nrow += 1
    plt.xlabel('Time (min)')
    plt.suptitle('Energy Consumption vs. Time')
    plt.savefig('Individual-Applications-Time-Series.png')

plotDisAggregatedTimeSeries(ts_dfs)

ValueError: ignored

In [0]:
precision_cv, recall_cv, f1_cv= performTimeSeriesCV(matrix, labels, 10)
print "Precision: %s, Recall: %s, F1 %s: for CV = 10" %(precision_cv, recall_cv, f1_cv)

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
print classification_report(y_testFold1, y_predsCV)